In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
import BPV
import pattern_manipulation as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import pickle

Wmin = 0.001
Wmax = 0.25
Wnsteps = 30
Wstep = (Wmax - Wmin)/Wnsteps
Wrange = np.arange(Wmin,Wmax,Wstep)

Nmin = 10
Nmax = 150
Nnsteps = 30
Nstep = (Nmax - Nmin)/Nnsteps
Nrange = np.arange(Nmin,Nmax,Nstep, dtype='int')

nproblems = Wnsteps*Nnsteps

data = BPV.Data()
data.read_csv("p.delviva.csv",False)
data.df.sort_index(by="p",inplace=True,ascending=True)
data.df.set_index(pd.Index([j for j in range(len(data.df))]), inplace=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
entropies = np.zeros((Wnsteps, Nnsteps))
rates = np.zeros((Wnsteps, Nnsteps))
cardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
indexes = {}

it = 1
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    W = Wrange[i]
    N = Nrange[j]
    print("Solving problem {0} out of {1}, for W = {2} and N = {3}".format(it, nproblems, W, N))
    
    prbl_exact = BPV.BPV("pulp",data,N,W,time_solver=False)
    prbl_exact.solve()
    #prbl_exact.pprint_solution()
    
    entropies[i,j] = prbl_exact.solution_entropy
    rates[i,j] = prbl_exact.solution_rate
    cardinalities[i,j] = prbl_exact.solution_cardinality
    indexes[(i,j)] = prbl_exact.solution.index
    it += 1
    
#f1 = open("/home/renato/tesi/code/plot6.entropies",r'wb')
#f2 = open("/home/renato/tesi/code/plot6.rates",r'wb')
#f3 = open("/home/renato/tesi/code/plot6.cardinalities",r'wb')
#f4= open("/home/renato/tesi/code/plot6.indexes",r'wb')

#pickle.dump(entropies,f1)
#pickle.dump(rates,f2)
#pickle.dump(cardinalities,f3)
#pickle.dump(indexes,f4)


Solving problem 1 out of 900, for W = 0.001 and N = 10
Solving problem 2 out of 900, for W = 0.001 and N = 14
Solving problem 3 out of 900, for W = 0.001 and N = 18
Solving problem 4 out of 900, for W = 0.001 and N = 22
Solving problem 5 out of 900, for W = 0.001 and N = 26
Solving problem 6 out of 900, for W = 0.001 and N = 30

In [5]:
#SAVE
f1 = open("/home/renato/tesi/code/plot6.entropies",r'wb')
f2 = open("/home/renato/tesi/code/plot6.rates",r'wb')
f3 = open("/home/renato/tesi/code/plot6.cardinalities",r'wb')
f4= open("/home/renato/tesi/code/plot6.indexes",r'wb')

pickle.dump(entropies,f1)
pickle.dump(rates,f2)
pickle.dump(cardinalities,f3)
pickle.dump(indexes,f4)


In [2]:
#LOAD
f1 = open("/home/renato/tesi/code/plot6.entropies",r'rb')
f2 = open("/home/renato/tesi/code/plot6.rates",r'rb')
f3 = open("/home/renato/tesi/code/plot6.cardinalities",r'rb')
f4 = open("/home/renato/tesi/code/plot6.indexes",r'rb')

entropies = pickle.load(f1)
rates = pickle.load(f2)
cardinalities = pickle.load(f3)
indexes = pickle.load(f4)

In [3]:
zeroindexes = (entropies == 0).nonzero()
zeroindexes = zip(zeroindexes[0], zeroindexes[1])
for i,j in zeroindexes:
    glpsol = BPV.BPV("glpk",data,Nrange[j],Wrange[i],time_solver=True)
    glpsol.solve()
    entropies[i,j] = glpsol.solution_entropy
    rates[i,j] = glpsol.solution_rate
    cardinalities[i,j] = glpsol.solution_cardinality
    indexes[(i,j)] = glpsol.solution.index

In [3]:
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(Wrange, Nrange)
ax.plot_surface(X, Y, entropies, rstride=1, cstride=1, alpha=0.5, cmap=cm.hsv)
ax.plot_wireframe(X, Y, rates, rstride=1, cstride=1, alpha=0.8)
#ax.plot_wireframe(X, Y, cardinalities/150, rstride=1, cstride=1, alpha=0.8)

#cset = ax.contour(X, Y, entropies, zdir='z', offset=-100, cmap=cm.coolwarm)
#cset = ax.contour(X, Y, entropies, zdir='x',  cmap=cm.coolwarm)
#cset = ax.contour(X, Y, entropies, zdir='y',  cmap=cm.coolwarm)

ax.set_xlabel('W')
ax.set_ylabel('N')
ax.set_zlabel('Entropy')

plt.show()

0.0093 58
0.0093 62
0.0425 74
0.2168 26
0.2251 22
0.2251 26
0.2334 22
0.2334 46


In [41]:
#prbl_exact = BPV.BPV("pulp",data,58,0.0093,time_solver=False)
#prbl_exact.write_cplex_lp("/home/renato/tesi/code/bug3.lp")
#prbl_exact.solve()
#prbl_exact.pprint_solution()

prbl_exact = BPV.BPV("heuristic",data,58,0.0093,time_solver=False)
prbl_exact.solve()
prbl_exact.pprint_solution()


Solver =  heuristic 
Entropy =  0.0764990388237 
Cardinality =  43 
Rate =  0.0090633272987


In [73]:
glpsol = BPV.BPV("glpk",data,58,0.0093,time_solver=True)
glpsol.solve()
glpsol.pprint_solution()

glpsol = BPV.BPV("pulp",data,58,0.0093,time_solver=True)
glpsol.solve()
glpsol.pprint_solution()


Solver =  glpk 
Entropy =  0.0810896093426 
Cardinality =  58 
Rate =  0.00929960810238
Solution time in seconds =  0.3107150880023255

Solver =  pulp 
Entropy =  0.0 
Cardinality =  0 
Rate =  0
Solution time in seconds =  0.2748125180005445


In [67]:
glpsol.solution

,pattern-string,pattern-matrix,p,plog1onp,heuristic,glpk
364,100111001,"[[1, 0, 0], [1, 1, 1], [0, 0, 1]]",0.000125,0.001122,False,True
366,110110100,"[[1, 1, 0], [1, 1, 0], [1, 0, 0]]",0.000126,0.001128,False,True
367,100110000,"[[1, 0, 0], [1, 1, 0], [0, 0, 0]]",0.000126,0.001128,False,True
368,100111100,"[[1, 0, 0], [1, 1, 1], [1, 0, 0]]",0.000126,0.001130,False,True
369,111000101,"[[1, 1, 1], [0, 0, 0], [1, 0, 1]]",0.000126,0.001132,False,True
370,110011001,"[[1, 1, 0], [0, 1, 1], [0, 0, 1]]",0.000127,0.001138,False,True
371,1110000,"[[0, 0, 1], [1, 1, 0], [0, 0, 0]]",0.000128,0.001146,False,True
372,100110011,"[[1, 0, 0], [1, 1, 0], [0, 1, 1]]",0.000129,0.001154,False,True
373,110100,"[[0, 0, 0], [1, 1, 0], [1, 0, 0]]",0.000129,0.001158,False,True
374,110001001,"[[1, 1, 0], [0, 0, 1], [0, 0, 1]]",0.000131,0.001168,False,True
